In [ ]:
'''
掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息素材準備
handler執行方法設計
啟動應用
'''

In [ ]:
'''
資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內
'''
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''

流程解析

'''


In [ ]:
!pip install line-bot-sdk flask flask-ngrok

In [ ]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [ ]:
'''
建置主程序app

建置handler與 line_bot_api
'''

# 設定Server啟用細節。準備 app 這個 flask HTTP Application
app = Flask(__name__, static_url_path = "/material" , static_folder = "./material/")
# 請 ngrok 給我們一個 https 臨時 URL
run_with_ngrok(app)

# 生成實體物件，跟line溝通用
line_bot_api = LineBotApi("your channel access token")
# 收發消息專用
handler = WebhookHandler("your channel secret")

In [ ]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value，取得用戶簽章
    signature = request.headers['X-Line-Signature']

    # get request body as text，取得用戶的消息
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log，把用戶傳來的消息紀錄下來
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body。把消息交給handler做處理
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
'''
消息素材準備
'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)
# xxxxSendMessage，回應用戶的消息物件
specific_text_message = TextSendMessage("官方建議，用戶以文字消息觸發特殊功能的時候，要加上特殊符號")

# 回應用戶的圖片消息，參數與line bot designer很類似。url需要是https網址，未來可以把圖片保存到cloud storage，會得到一個https網址
analyze_truth_image_message = ImageSendMessage(
      original_content_url='https://i.imgur.com/M8jnn9B.png',
      preview_image_url='https://i.imgur.com/LtCtgg0.jpg'
      )

In [ ]:
'''
設計一個字典，當用戶輸入相應文字消息時，系統根據字典回應消息。未來很可能是資料庫，把要回應的消息存在資料庫，從資料庫查找回應訊息
'''

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
  "@123": specific_text_message,
  "@分析的本質": analyze_truth_image_message
}

In [ ]:
# 用戶發出文字消息時， 按條件內容, 回傳文字消息。用戶傳來的文字建議加上特殊符號
# MessageEvent 是消息事件，有文字消息 TextMessage，圖片消息 ImageMessage，影片消息 VideoMessage 等。最多發五則消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    if(event.message.text.find('@')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    else:
        line_bot_api.reply_message(
        event.reply_token,         # 回郵信函，有時效性
        TextSendMessage(text="字典內查無此字，請輸入@123或@分析的本質")
        )
# Text Message 是用戶傳來的消息，TextSendMessage 是我們要傳給用戶的消息

In [ ]:
# 主程序運行
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://0ae7-35-221-229-103.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"destination":"Ude2ebddaef1e1b50ddbe960f037b8efe","events":[{"type":"message","message":{"type":"text","id":"16910292665053","text":"@123"},"webhookEventId":"01GERHKJFK22KP72GTX0BX7WVV","deliveryContext":{"isRedelivery":false},"timestamp":1665123600511,"source":{"type":"user","userId":"U64e856315f80175e21257c6e7eb77412"},"replyToken":"dc1dbea77dbc4fa28ebd91a181c134fc","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [07/Oct/2022 06:20:02] "POST / HTTP/1.1" 200 -
ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-5-db5f98947edd>", line 26, in callback
 

{"destination":"Ude2ebddaef1e1b50ddbe960f037b8efe","events":[{"type":"message","message":{"type":"text","id":"16910293135642","text":"@我"},"webhookEventId":"01GERHKRHD4XW4CXQR0TB9KTAN","deliveryContext":{"isRedelivery":false},"timestamp":1665123606944,"source":{"type":"user","userId":"U64e856315f80175e21257c6e7eb77412"},"replyToken":"77c3e92d32de47fc97d2115a61d9d082","mode":"active"}]}


...

{"destination":"Ude2ebddaef1e1b50ddbe960f037b8efe","events":[]}


ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-5-db5f98947edd>", line 26, in callback
    handler.handle(body, signature)
  File "/usr/local/lib/python3.7/dist-pa

{"destination":"Ude2ebddaef1e1b50ddbe960f037b8efe","events":[{"type":"message","message":{"type":"text","id":"16910296433554","text":"@轻微"},"webhookEventId":"01GERHN4SWNXT231E8QMDVCX9N","deliveryContext":{"isRedelivery":false},"timestamp":1665123651985,"source":{"type":"user","userId":"U64e856315f80175e21257c6e7eb77412"},"replyToken":"9f32b0c17fca4544b96f7e2e2030716f","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [07/Oct/2022 06:21:04] "POST / HTTP/1.1" 200 -


{"destination":"Ude2ebddaef1e1b50ddbe960f037b8efe","events":[{"type":"message","message":{"type":"text","id":"16910297249397","text":"@123"},"webhookEventId":"01GERHNFM92HEDC703BQ64ZWT5","deliveryContext":{"isRedelivery":false},"timestamp":1665123663124,"source":{"type":"user","userId":"U64e856315f80175e21257c6e7eb77412"},"replyToken":"099942b844d34dffaab2e554b5c8b4b7","mode":"active"}]}
